## 🏡 **Build the HomeMatch application**

#### Understanding Buyer Preferences:
 - Buyers will input their requirements and preferences, such as location, property type, budget, amenities, and lifestyle choices.

 - The application uses LLMs to interpret these inputs in natural language, understanding nuanced requests beyond basic filters.

#### Integrating with a Vector Database:
 - Connect "HomeMatch" with a vector database, where all available property listings are stored.
 
 - Utilize vector embeddings to match properties with buyer preferences, focusing on aspects like neighborhood vibes, architectural styles, and proximity to specific amenities.

#### Personalized Listing Description Generation:
- For each matched listing, use an LLM to rewrite the description in a way that highlights aspects most relevant to the buyer’s preferences.

- Ensure personalization emphasizes characteristics appealing to the buyer without altering factual information about the property.

#### Listing Presentation:

- Output the personalized listing(s) as a text description of the listing.

## 📍 **Step 1: Setting Up the Python Application**

In [21]:
import csv
import pandas as pd
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain

In [7]:
import openai

openai.api_key = "YOUR_API_KEY"

# Define OpenAI Models
COMPLETION_MODEL = 'gpt-3.5-turbo'

# Initialize OpenAI Chat model
llm = OpenAI(
    openai_api_key=openai.api_key, 
    model=COMPLETION_MODEL, 
    temperature=0.5,
    max_tokens=50
)

## 📍 **Step 2: Generating Real Estate Listings**

In [5]:
prompt_template = '''
Generate a real estate listing with the following details:
Neighborhood: {Neighborhood}
Price: {Price}
Bedrooms: {Bedrooms}
Bathrooms: {Bathrooms}
House Size: {House_Size}

Description: Write a detailed and appealing description of the property.

Neighborhood Description: Write a description of the neighborhood including nearby amenities and the community atmosphere.
'''

input_variables = {
    'Neighborhood': "Neighborhood",
    'Price': "Price (k)",
    'Bedrooms': "Bedrooms",
    'Bathrooms': "Bathrooms",
    'House_Size': "House_Size (sqft)",
}

In [14]:
rows = "10"
generated_listings = []

for _ in range(int(rows)):
    formatted_prompt = prompt_template.format(**input_variables).strip()
    
    response = openai.chat.completions.create(
        model=COMPLETION_MODEL,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": formatted_prompt}
        ],
        temperature=0,
        max_tokens=50
    )

    response_message = response.choices[0].message.content
    generated_listings.append(response_message)
    print(response_message)

🏡 Real Estate Listing 🏡

Neighborhood: Sunnyvale Heights
Price: $950k
Bedrooms: 3
Bathrooms: 2.5
House Size: 2,000 sqft

Description:

🏡 Real Estate Listing 🏡

Neighborhood: Serene Hills
Price: $750k
Bedrooms: 4
Bathrooms: 3
House Size: 2,500 sqft

Description:
Welcome to
🏡 Real Estate Listing 🏡

Neighborhood: Serene Valley
Price: $500k
Bedrooms: 3
Bathrooms: 2
House Size: 2,000 sqft

Description:
Welcome to
🏡 Real Estate Listing 🏡

Neighborhood: Serene Hills
Price: $750k
Bedrooms: 4
Bathrooms: 3
House Size: 2,500 sqft

Description:
Welcome to
🏡 Real Estate Listing 🏡

Neighborhood: Serene Valley
Price: $500k
Bedrooms: 3
Bathrooms: 2
House Size: 2,000 sqft

Description:
Welcome to
🏡 Real Estate Listing 🏡

Neighborhood: Serene Valley
Price: $500k
Bedrooms: 3
Bathrooms: 2
House Size: 2,000 sqft

Description:
Welcome to
🏡 Real Estate Listing 🏡

Neighborhood: Serene Valley
Price: $450k
Bedrooms: 3
Bathrooms: 2.5
House Size: 2,000 sqft

Description:

🏡 Real Estate Listing 🏡

Neighborhood: Se

### ➡︎ **Load the dataset**

In [18]:
output_file = 'generated_listings.csv'

with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Generated Real Estate Listings'])
    writer.writerow([])

    for listing in generated_listings:
        writer.writerow([listing])

print(f"Generated listings saved to {output_file}")

Generated listings saved to generated_listings.csv


In [24]:
import csv
import re

def extract_features(text):
    features = {}
    patterns = {
        'Neighborhood': r'Neighborhood: (.+)',
        'Price': r'Price: \$(.+)k',
        'Bedrooms': r'Bedrooms: (\d+)',
        'Bathrooms': r'Bathrooms: ([\d\.]+)',
        'House_Size': r'House Size: ([\d,]+) sqft'
    }
    
    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            features[key] = match.group(1)
    
    return features

listings = []
with open('generated_listings.csv', 'r') as file:
    content = file.read()
    listings = content.split('"🏡 Real Estate Listing 🏡\n\n')[1:]  # 分割每個列表項

extracted_features = [extract_features(listing) for listing in listings]

with open('extracted_features.csv', 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['Neighborhood', 'Price', 'Bedrooms', 'Bathrooms', 'House_Size'])
    writer.writeheader()
    for feature in extracted_features:
        writer.writerow(feature)

In [3]:
df = pd.read_csv("extracted_features.csv")
df.head()

,Neighborhood,Price (k),Bedrooms,Bathrooms,House_Size (sqft)
0,Sunnyvale Heights,950,3,2.5,"2,000"
1,Serene Hills,750,4,3.0,"2,500"
2,Serene Valley,500,3,2.0,"2,000"
3,Serene Hills,750,4,3.0,"2,500"
4,Serene Valley,500,3,2.0,"2,000"


## 📍 **Step 3: Storing Listings in a Vector Database**

In [31]:
import os
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

In [32]:
df = CSVLoader(file_path='extracted_features.csv').load()
llm = ChatOpenAI(model_name=COMPLETION_MODEL, temperature=0.5, max_tokens=50)

In [33]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(df)

In [34]:
embeddings = OpenAIEmbeddings()
vector_store = FAISS.from_documents(texts, embeddings)

## 📍 **Step 4: Building the User Preference Interface**

In [35]:
questions = [
    "What's your preferred price range for the property?",
    "How many bedrooms and bathrooms are you looking for?",
    "Which neighborhood are you most interested in?",
    "What's the minimum house size you're comfortable with?",
]

answers = [
    "I'm looking for something in the $500k to $750k range.",
    "I need at least 3 bedrooms and 2 bathrooms.",
    "I'm particularly interested in the Serene Valley area.",
    "I'd like a house that's at least 2,000 square feet.",
]

history = ChatMessageHistory()
history.add_user_message("I'm looking for a new home. Can you help me find one?")
history.add_ai_message("Of course! I'd be happy to help you find a new home. Let me ask you a few questions to understand your preferences better.")

In [36]:
buyer_preferences = {}
for question, answer in zip(questions, answers):
    history.add_ai_message(question)
    history.add_user_message(answer)
    buyer_preferences[question] = answer

memory = ConversationSummaryMemory(
    llm=ChatOpenAI(temperature=0),
    chat_memory=history,
    memory_key="chat_history",
    return_messages=True
)

## 📍 **Step 5: Searching Based on Preferences**

In [37]:
def semantic_search(vector_store, buyer_preferences):
    query = " ".join([f"{k}: {v}" for k, v in buyer_preferences.items()])
    relevant_listings = vector_store.similarity_search(query, k=3)
    return relevant_listings

## 📍 **Step 6: Personalizing Listing Descriptions**

In [38]:
def personalize_listing(listing, buyer_preferences, chat_history):
    llm = ChatOpenAI(temperature=0.7)
    prompt = PromptTemplate(
        input_variables=["listing", "preferences", "chat_history"],
        template="""
        Given the following real estate listing, buyer preferences, and chat history, enhance the listing description to resonate with the buyer's needs. 
        Maintain factual accuracy while highlighting aspects that align with the buyer's preferences and addressing any concerns or interests mentioned in the chat history.

        Listing: {listing}
        Buyer Preferences: {preferences}
        Chat History: {chat_history}

        Enhanced listing description:
        """
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    enhanced_description = chain.run(listing=listing, preferences=str(buyer_preferences), chat_history=chat_history)
    return enhanced_description

In [39]:
def find_and_personalize_listings(vector_store, buyer_preferences, memory):
    relevant_listings = semantic_search(vector_store, buyer_preferences)
    chat_history = memory.load_memory_variables({})["chat_history"]
    
    personalized_listings = []
    for listing in relevant_listings:
        enhanced_description = personalize_listing(listing.page_content, buyer_preferences, chat_history)
        personalized_listings.append(enhanced_description)
    return personalized_listings


In [40]:
recommend_prompt = PromptTemplate(
    template="You are a sales assistant who recommends homes. Use the following pieces of retrieved context, customer preferences, and chat history to provide the customer with information about available homes. Use five sentences maximum and keep the answer attractive. \nContext: {context} \nCustomer's preferences: {chat_history} \nQuestion: {question}\nAnswer:",
    input_variables=['context', 'chat_history', 'question']
)

chain = ConversationalRetrievalChain.from_llm(
    llm=ChatOpenAI(temperature=0.3),
    retriever=vector_store.as_retriever(),
    combine_docs_chain_kwargs={'prompt': recommend_prompt},
    memory=memory
)

In [41]:
initial_query = "Based on our conversation, what homes would you recommend for me?"
initial_result = chain({"question": initial_query})

print("Initial recommendation:")
print(initial_result['answer'])
print("\n---\n")

/Users/work/udacity-generative-ai/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Initial recommendation:
I would recommend two homes that match your preferences. In Serene Hills, there is a beautiful 4-bedroom, 3-bathroom home with 2,500 sqft priced at $750k. Additionally, in Sunnyvale Heights, there is a lovely 3-bedroom, 2.5-bathroom home with 2,000 sqft priced at $950k. Both homes offer a serene environment and spacious living areas, perfect for your needs.

---



In [42]:
personalized_results = find_and_personalize_listings(vector_store, buyer_preferences, memory)

print("Personalized listings based on your preferences and our conversation:")
for i, result in enumerate(personalized_results, 1):
    print(f"\nListing {i}:")
    print(result)
    print("---")

Personalized listings based on your preferences and our conversation:

Listing 1:
Welcome to Serene Valley, the perfect neighborhood that meets all your criteria! This charming home offers 3 bedrooms and 2 bathrooms, with a spacious 2,000 square feet of living space, all within your preferred price range of $500k to $750k.

Nestled in the serene surroundings of Serene Valley, this property provides the peaceful environment you desire. With a comfortable house size of 2,000 square feet, this home offers ample space for your family to relax and enjoy.

Don't miss out on this opportunity to own your dream home in the sought-after Serene Valley area. Contact us today to schedule a viewing and make this beautiful property yours!
---

Listing 2:
Welcome to Serene Valley, the neighborhood of your dreams! This charming home features 3 bedrooms and 2 bathrooms, perfectly fitting your requirements. With a spacious 2,000 square feet of living space, this property offers ample room for you and you